# Data exploration of House Price Predictions


The data is downloaded from Kaggle and contains data about houses and around ~80 variables. The goal is to use these explanatory variables to predict the House Prices. Here, we are dealing with a regression problem. As we want to conduct a classification, as well, we will also divide the SalePrice in three categories, namely "low", "middle" and "upper class". We will then attempt to predict these 3 classes. 

https://www.kaggle.com/c/house-prices-advanced-regression-techniques

This notebook is to explore the data, to understand the basic relationships between the variables and to get a feeling about which variables might be good predictors for the House prices. There will be a separate notebook containing statistical and machine learning models for the predictions.

Author: Julia Hammerer, Vanessa Mai
Last Changes: 18.11.2018

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-exploration-of-House-Price-Predictions" data-toc-modified-id="Data-exploration-of-House-Price-Predictions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data exploration of House Price Predictions</a></span><ul class="toc-item"><li><span><a href="#Data-Profile" data-toc-modified-id="Data-Profile-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Data Profile</a></span></li><li><span><a href="#Missing-values" data-toc-modified-id="Missing-values-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Missing values</a></span></li><li><span><a href="#Sanity-checks" data-toc-modified-id="Sanity-checks-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Sanity checks</a></span></li><li><span><a href="#Cleanse-data" data-toc-modified-id="Cleanse-data-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Cleanse data</a></span></li><li><span><a href="#Explorations" data-toc-modified-id="Explorations-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Explorations</a></span><ul class="toc-item"><li><span><a href="#Correlation-Analysis-for-numeric-features" data-toc-modified-id="Correlation-Analysis-for-numeric-features-1.5.1"><span class="toc-item-num">1.5.1&nbsp;&nbsp;</span>Correlation Analysis for numeric features</a></span></li><li><span><a href="#Categorical-Features" data-toc-modified-id="Categorical-Features-1.5.2"><span class="toc-item-num">1.5.2&nbsp;&nbsp;</span>Categorical Features</a></span></li></ul></li><li><span><a href="#Explorations-based-on-self-defined-categories" data-toc-modified-id="Explorations-based-on-self-defined-categories-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Explorations based on self-defined categories</a></span></li></ul></li></ul></div>

In [ ]:
import sys
sys.path.insert(0, '../helper/')

In [ ]:
# load packages
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import pandas_profiling
import missingno as msno
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import warnings
import math

from pandas.tools.plotting import table
from plotly.offline import init_notebook_mode
from plotly.offline import iplot
from plotly.offline import plot
from scipy.stats import mannwhitneyu
from statsmodels.distributions.empirical_distribution import ECDF
from scipy import stats
from scipy.stats import pearsonr
from scipy.stats import norm

from helper import na_ratio_table
from helper import corr_heatmap
from helper import corr_matrix_1


In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
#load data
# we have two files, since this is a part of a kaggle competition,
# only the training-set contains the target variable
# we will use that for the whole analysis

df=pd.read_csv("../data/house_prices_train.csv")

In [ ]:
print("Number of records and variables: ",df.shape)

## Data Profile

In [ ]:
# for a first overview, we apply the pandas-profile report
# it provides simple histograms, distributions, missingness 
# and correlations for all variables

pandas_profiling.ProfileReport(df)

A description of all data fields can be found on the Kaggle site: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data. Most of them are self-explanatory though.

Around half the variables are categorical and the other half are numerical. For the categorical variables there will be need to use hot-one-encoding for incorporating them into the prediction models.

We can already spot some correlations that look promising. Some of them
also are also expected and won't give us further insights. We are particularly
interested in correlations with our target variable
- OverallQual - SalePrice
- GrLivArea - SalePrice
- FullBath - SalePrice
- GarageYrBlt - YearRemodAdd
- LotFrontage - lotArea
- TotRmsAbvGrd - GrLivArea
- BsmtUnfSF - BsmtFinSF1: negative correlation

Also, we can detect variables that probably won't be of much use
e.g.
- Street: only two values, of which one is extremly low.
- Utilities: Almost constant with two values, of which the other one has only one record

In [ ]:
df.select_dtypes(include=['object']).head()

## Missing values
Let's check the missingness in more detail

In [ ]:
display(na_ratio_table(df)[na_ratio_table(df)["NA_COUNT"]>0])
display(na_ratio_table(df)[na_ratio_table(df)["NA_COUNT"]>0].shape)


We have 19 variables that contain missing values. Most of them mean that the feature is simply not available for that property. However for a few, this can indicate a data quality issue:
- Electrical: the type is not stated, it is improbable that there is no electrical system at all. 
- LotFrontage: a building should always have a lotfrontage

As for "Electrical" only one record is missing, we can simply filter this out, or even ignore this. For the LotFrontage we can apply some imputation-techniques if necessary.

In [ ]:
# we test if the data is randomly missing, or if there are some patterns in the missingness
# this helps us indicate whether there are data quality issues or if the missingness is part of the data
msno.heatmap(df)

As expected, we can see that some of the variables are always missing together, which makes absolutely sense. 
Example: All Garage related variables are always missing together. Reason: no garage -> no values for any garage features.
The other group of variables missing together is related to the basement. Because of these correlations, we might run into some multicollinearity issues in the modelling part later on.  "Multicollinearity is a state of very high intercorrelations or inter-associations among the independent variables. It is therefore a type of disturbance in the data, and if present in the data the statistical inferences made about the data may not be reliable." (https://www.statisticssolutions.com/multicollinearity/, Accesed on: 22.11.2018) <br>


## Sanity checks
We're going to check if there are some inconsistencies in the data or duplicates, etc. (Quality assessment)

In [ ]:
# any duplicates?
df[df.duplicated(keep=False)]


In [ ]:
# any built year before sold year?
df.query('YearBuilt > YrSold')


## Cleanse data

In [ ]:
# remove useless columns
df=df.drop(columns=["Id", "Street", "Utilities"])

In [ ]:
# remove the missing record for Electrical
df=df[df["Electrical"].isna()==False]

## Explorations

In [ ]:
df.describe()

Let's once again check the distribution of the SalePrice

In [ ]:
sns.distplot(df["SalePrice"], hist=False, label="SalePrice", fit=norm,  kde_kws={"shade": True})

The distribution is slightly skewed. 

In [ ]:
stats.probplot(df['SalePrice'], plot=plt)

SalePrice does not have a normal distribution.

### Correlation Analysis for numeric features

In [ ]:
# we check for further correlations using different plots
NUM_FEATURES =df.select_dtypes(include=[np.number]).columns.tolist()

df_num=df[NUM_FEATURES]

df_corr=df_num.corr()

In [ ]:
corr_heatmap(df_corr, figsize=(20, 16))

In [ ]:
corr_matrix_1(df_corr)

This visualization is not suitable due to the great amount of variables. Let's filter out the ones with the highest correlations and visualize those.

In [ ]:
df_corr=df_corr.abs()

df_corr=(df_corr.where(~np.tril(np.ones(df_corr.shape)).astype(np.bool)))

df_high_corr=(df_corr[df_corr>0.75].dropna(how="all", axis=0).dropna(how="all", axis=1))

df_high_corr_vars=np.unique(np.concatenate((df_high_corr.columns.values, df_high_corr.index.values)))

df_high_corr = df[df_high_corr_vars]

df_high_corr.shape

In [ ]:
corr_matrix_1(df_high_corr)

In [ ]:
df_corr.iloc[:,-1].sort_values(ascending=False)[:10]

The variables which seem to have a high correlation with the target variable are also amongst the ones with the highest correlations in general.
Let's also have a look at some of the categorical variables. 

### Categorical Features

In [ ]:
CAT_FEATURES =df.select_dtypes(include="object").columns.tolist()

In [ ]:
plt.figure(figsize=(40,400))

for i, a in enumerate(CAT_FEATURES):
#     plt.figure(figsize=(5,2))
    plt.subplot(math.ceil(len(CAT_FEATURES)),2,((i+1)*2-1))
    sns.boxplot(x="SalePrice", y=a, data=df)
    plt.ylabel(a, fontsize=40)
    plt.tick_params(axis='y', which='major', labelsize=30)

    plt.subplot(math.ceil(len(CAT_FEATURES)),2,((i+1)*2))
    for b in df[a].unique():
        sns.distplot(df[df[a]==b]["SalePrice"], hist=False, label=a,  kde_kws={"shade": True})
#     plt.ylabel(a, fontsize=40)

#     plt.tight_layout()
# sns.distplot(tsh_sta_pcu.query('Label_trunc == "PCU1"')['Wdf'], hist=False, label='PCU1', kde_kws={"shade": True})
# sns.distplot(tsh_sta_pcu.query('Label_trunc == "PCU2"')['Wdf'], hist=False, label='PCU2', kde_kws={"shade": True});

This gives us a feeling of which attributes might be more important and can help us predict the SalePrice.
For these attributes for example the SalePrice is different based on the value:
* Alley
* Neighborhood
* ExterQual
* ExterCond
* KitchenQual
* PoolQual

For BsmtFinType2 e.g. the SalePrice cannot be differentiated much. Hence, this attribute might have less predictive power compared to some of the other attriutes.

The EDA here already shows us potential to predict the SalePrice. Definitively, there are some relationship towards the SalePrice. Next step is to built models to predict the SalePrice and investigate the significance of differences in the attributes.

## Explorations based on self-defined categories
As mentioned in the beginning, we will also divide the SalePrice in three categories, namely "low", "middle" and "upper class". Thus, we also want to see how these three can be separated and if there is some obvious differences. 

In [ ]:
sns.distplot(df["SalePrice"], hist=True, label="SalePrice", fit=norm,  kde_kws={"shade": True})

In [ ]:
df["SalePrice"].describe()

In [ ]:
df["SalePrice"].median()

In [ ]:
plt.hist(df.SalePrice, bins=10)

The range of the SalePrice is from 34,900 - 755,000. The middle class would be the most frequent one. We decided to use the following boundaries:
- low: 0-100,000
- middle: 100,001-400,000
- upper: from 400,001 onwards

In [ ]:
# create new column
def price_class(row):
    if row["SalePrice"]<=120000:
        return "low"
    elif (row["SalePrice"]>120000)& (row["SalePrice"]<=250000):
        return "middle"
    else:
        return"upper"


df["Price_Class"]= df.apply(lambda row: price_class(row), axis=1)

In [ ]:
plt.figure(figsize=(40,400))

for i, a in enumerate(NUM_FEATURES):
#     plt.figure(figsize=(5,2))
    plt.subplot(math.ceil(len(CAT_FEATURES)),2,((i+1)*2-1))
    sns.boxplot(y=a, x="Price_Class", data=df)
    plt.ylabel(a, fontsize=40)
    plt.tick_params(axis='y', which='major', labelsize=30)

    plt.subplot(math.ceil(len(CAT_FEATURES)),2,((i+1)*2))
    for b in df["Price_Class"].unique():
        sns.distplot(df[df["Price_Class"]==b][a], hist=False, label=b,  kde_kws={"shade": True})
#     plt.ylabel(a, fontsize=40)

#     plt.tight_layout()
# sns.distplot(tsh_sta_pcu.query('Label_trunc == "PCU1"')['Wdf'], hist=False, label='PCU1', kde_kws={"shade": True})
# sns.distplot(tsh_sta_pcu.query('Label_trunc == "PCU2"')['Wdf'], hist=False, label='PCU2', kde_kws={"shade": True});